In [35]:
import pandas as pd
import os
import json

class Probe(object):
    def __init__(self, data_path, knownColumns = False):
        self.data_path = data_path
        ret = self.ensure_file(self.data_path)
        self.data = pd.read_csv(self.data_path, sep='\t', engine='python', header=None, skiprows=1)
        self.label_columns(knownColumns = knownColumns)
        self.averages = self.get_averages()
        
    @staticmethod
    def ensure_file(path):
        if os.path.isfile(path):
            return True
        else:
            print("Error: file {} does not exist".format(path))
            return False
    
    def get_averages(self):
        #TODO
        
        # group data by angle
        # in each of 5 pressure columns, take average over one angle and 15 sec
        
        # normalize by drift data
        # take tunnel_ref_total_pressure, uut_total_pressure (unit under test), take average of each,
        # take difference to find offset to apply to all other uut_total_pressure data (for all angles)
        
        # compare both static pressures to the tunnel_ref_static_pressure and do the same as above
        
        # find impact pressure = tunnel_total - tunnel_static (qc) (do for each angle, do )
        # divide all uut pressure normalized averages (one total, two static) by impact pressure (qc) for that angle
        
        
        return None
    
    def label_columns(self, knownColumns = False):
        if self.ensure_file('knownHeaders.txt'):
            #print('file exists')
            with open('knownHeaders.txt') as infile:
                allFormats = json.load(infile)
            selected = False
            while selected == False:
                print('Known headers:')
                for key in allFormats.keys():
                    print(key)
                name = input("Choose an existing header or type NO:")
                if name == 'NO':
                    selected = True
                else:
                    try:
                        colDict = allFormats[name]
                        print(colDict)
                        knownColumns=True
                        selected = True
                    except:
                        print('Unknown header.')
        else:
            allFormats = {}
        if knownColumns:
            #print('applying preselected column headers')
            self.known_columns(colDict)
        else:
            colDict = self.unknown_columns()
            #print(colDict)
            name = input("What would you like to save this header as? If you do not want to save, type NO.")
            if name == 'NO':
                pass
            else:
                allFormats[name] = colDict
                with open('knownHeaders.txt', 'w') as outfile:
                    json.dump(allFormats, outfile)
        return None
    
    def unknown_columns(self):
        # query the user to determine which column is which
        colDict = {}
        print(self.data.head())
        cols = {
            'angle': 'angle', 
            'tunnel reference total pressure': 'tunnel_ref_total_pressure', 
            'unit under test total pressure': 'uut_total_pressure',
            'tunnel reference static pressure': 'tunnel_ref_static_pressure',
            'unit under test static pressure 1': 'uut_static_pressure1',
            'unit under test static pressure 2': 'uut_static_pressure2',
            'date': 'date',
            'time': 'time',
            'temperature in F': 'tempF'
        }
        for key in sorted(cols.keys()):
            col = input("Which column number contains the {}?".format(key))
            col = int(col)
            self.data.rename(index=str, columns={col: cols[key]}, inplace=True)
            colDict[col]=cols[key]
            print('')
            print(self.data.head())
        return colDict
    
    def known_columns(self, colDict):
        for key in colDict.keys():
            self.data.rename(index=str, columns={int(key): colDict[key]}, inplace=True)
        return None
    
class AeroRepeatability(object):
    def __init__(self, golden_probe, uut_probe):
        self.golden_probe = golden_probe
        self.uut_probe = uut_probe
    
    def compare_static(self):
        #TODO
        return
    
    def compare_total(self):
        #TODO
        return
        
###
# after have averages compare this way:
#

In [ ]:
# user sequence:
golden_probe_path = input('Path to golden probe data: ')
uut_probe_path = input('Path to uut probe data: ')
AeroRepeatability(golden_probe_path, uut_probe_path)

In [3]:
import os

In [2]:
golden_probe_path = '/home/stoker/aero_analysis/A0251-50kn'
uut_probe_path = '/home/stoker/aero_analysis/SN003-50kn'

In [33]:
golden_probe = Probe(golden_probe_path)
#uut_probe = Probe(uut_probe_path)

Known headers:
wrong
test1
Choose an existing header or type NO:test1
{'6': 'tunnel_ref_static_pressure', '0': 'date', '3': 'uut_static_pressure1', '1': 'time', '5': 'uut_total_pressure', '2': 'tunnel_ref_total_pressure', '9': 'tempF', '4': 'uut_static_pressure2', '10': 'angle'}
applying preselected column headers
6
tunnel_ref_static_pressure
0
date
3
uut_static_pressure1
1
time
5
uut_total_pressure
2
tunnel_ref_total_pressure
9
tempF
4
uut_static_pressure2
10
angle


In [34]:
golden_probe.data.head()

,date,time,tunnel_ref_total_pressure,uut_static_pressure1,uut_static_pressure2,uut_total_pressure,tunnel_ref_static_pressure,7,8,tempF,angle
0,6/6/2018,4:02:08 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
1,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
2,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
3,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
4,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data


In [36]:
data = golden_probe.data

In [37]:
data.head()

,date,time,tunnel_ref_total_pressure,uut_static_pressure1,uut_static_pressure2,uut_total_pressure,tunnel_ref_static_pressure,7,8,tempF,angle
0,6/6/2018,4:02:08 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
1,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
2,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
3,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data
4,6/6/2018,4:02:09 PM,14.619227,14.625174,14.61524,14.669576,14.6177,14.623294,14.621277,70.0,Drift Data


In [38]:
data.tail()

,date,time,tunnel_ref_total_pressure,uut_static_pressure1,uut_static_pressure2,uut_total_pressure,tunnel_ref_static_pressure,7,8,tempF,angle
1770,6/6/2018,4:24:23 PM,14.626356,14.565466,14.629299,14.630360,14.566641,14.564556,14.625360,70.0,0856LU6-AI A0251 ATP at +20 deg AOA
1771,6/6/2018,4:24:24 PM,14.627355,14.565466,14.629299,14.629358,14.566641,14.564556,14.626358,70.0,0856LU6-AI A0251 ATP at +20 deg AOA
1772,6/6/2018,4:24:24 PM,14.627355,14.565466,14.629299,14.630360,14.566641,14.565553,14.625360,70.0,0856LU6-AI A0251 ATP at +20 deg AOA
1773,6/6/2018,4:24:24 PM,14.627355,14.566464,14.629299,14.630360,14.566641,14.565553,14.625360,70.0,0856LU6-AI A0251 ATP at +20 deg AOA
1774,6/6/2018,4:24:24 PM,14.627355,14.565466,14.629299,14.630360,14.566641,14.564556,14.625360,70.0,0856LU6-AI A0251 ATP at +20 deg AOA


In [39]:
# get averages
# parse angles
data['angle'].value_counts()
# group data by angle
        # in each of 5 pressure columns, take average over one angle and 15 sec
#data.groupby
        # normalize by drift data
        # take tunnel_ref_total_pressure, uut_total_pressure (unit under test), take average of each,
        # take difference to find offset to apply to all other uut_total_pressure data (for all angles)
        
        # compare both static pressures to the tunnel_ref_static_pressure and do the same as above
        
        # find impact pressure = tunnel_total - tunnel_static (qc) (do for each angle, do )
        # divide all uut pressure normalized averages (one total, two static) by impact pressure (qc)

0856LU6-AI A0251 ATP  at 0 deg AOA      152
 Drift Data                              88
0856LU6-AI A0251 ATP  at -6 deg AOA      80
0856LU6-AI A0251 ATP  at -14 deg AOA     78
0856LU6-AI A0251 ATP  at -10 deg AOA     78
0856LU6-AI A0251 ATP  at -4 deg AOA      78
0856LU6-AI A0251 ATP  at -18 deg AOA     78
0856LU6-AI A0251 ATP  at +8 deg AOA      78
0856LU6-AI A0251 ATP  at -2 deg AOA      78
0856LU6-AI A0251 ATP  at -16 deg AOA     78
0856LU6-AI A0251 ATP  at +2 deg AOA      78
0856LU6-AI A0251 ATP  at +4 deg AOA      78
0856LU6-AI A0251 ATP  at -12 deg AOA     77
0856LU6-AI A0251 ATP  at +12 deg AOA     77
0856LU6-AI A0251 ATP  at +20 deg AOA     77
0856LU6-AI A0251 ATP  at -8 deg AOA      77
0856LU6-AI A0251 ATP  at +6 deg AOA      76
0856LU6-AI A0251 ATP  at +18 deg AOA     76
0856LU6-AI A0251 ATP  at +14 deg AOA     76
0856LU6-AI A0251 ATP  at +10 deg AOA     76
0856LU6-AI A0251 ATP  at +16 deg AOA     74
0856LU6-AI A0251 ATP  at -20 deg AOA     67
Name: angle, dtype: int64